# 📌 민원 텍스트 분류 및 정부 부처 추천 시스템

---

## 🧾 프로젝트 정보

- **진행기간**: 2025.03 ~ 진행 중  
- **참여 형태**: 개인 프로젝트 (기획, 전처리, 모델링, 배포까지 단독 수행)  
- **프로젝트 개요**:  
  사용자가 입력한 민원 텍스트를 18개 카테고리로 분류한 뒤, 예측된 카테고리에 맞는 중앙정부 부처의 **홈페이지 및 전화번호를 자동으로 제공**하는 시스템을 개발.
  전이학습 기반 NLP 모델을 통해 민원 카테고리를 분류하고 예측 결과를 바탕으로 관련 부처의 정보(홈페이지, 전화번호)를 제공하는 **MCP(Model Context Protocol)** 알고리즘 설계.

---

## ✅ 최종 결과

- **최종 모델**: `beomi/KcELECTRA-base`
- **정확도**: 94.01%
- **Precision / Recall / F1 Score**: 약 93% 이상 (Macro 평균 기준)

---

## 📂 데이터셋 정보
- **출처**: AI-Hub의 민원 업무화 인공지능 언어 데이터 사용
- **형식**: JSON (`원천데이터`, `라벨링데이터`로 분리 제공)
- **구성요소**:
  - 원천데이터: `id`, `text`
  - 라벨링데이터: `id`, `category`, `subcategory`, `prediction`, `department`, `Q_refined`... 다수
- **데이터 형태 변환**
  - 원천데이터_train/val.csv
  - 라벨링데이터_train/val.csv
- **최종 학습셋**(데이터 전처리)
  - 민원통합데이터_train.csv
  - 민원통합데이터_val.csv
---

## 🛠 사용 기술

### 📦 프레임워크 & 라이브러리
- Transformers (HuggingFace)  
- PyTorch  
- Pandas / Numpy  
- Matplotlib / Seaborn  
- Scikit-learn  
- Flask

### 💻 개발 환경
- Python 3.x  
- Jupyter Notebook  
- VSCode  
- Google Colab (T4 GPU)  
- AWS EC2 (배포 예정)

### 🧠 모델링 기법
- 전이학습 (Transfer Learning)  
- BERT 계열 사전학습 모델 활용 (KoBERT, KcBERT, KcELECTRA)  
- Fine-tuning + EarlyStopping  
- TF-IDF + LightGBM (Baseline)

---

## 🧩 트러블 슈팅 및 개선 과정

1. **데이터 병합**

<table style="margin: 0; margin-bottom: 20px;">
  <tr>
    <td><img src="https://john-yim.github.io/portfolio/images/complain_id_venn.png" style="width: 700px; height: auto;"></td>
  </tr>
  <tr>
    <td style="text-align: center;"><i>train, val 데이터 고유 id 벤다이어그램</i></td>
  </tr>
</table>

   - 학습 데이터와 검증 데이터 간 `id` 매칭 불일치
   - `train`에는 있는데 `val`에는 없는 샘플 / 반대도 존재
   → `inner join`으로 정합성 확보

2. **데이터 중복 문제**
   - 동일한 `text`를 가진 데이터 존재

<table style="margin: 0; margin-bottom: 20px;">
  <tr>
    <td><img src="https://john-yim.github.io/portfolio/images/text_dup.png" style="width: 700px; height: auto;"></td>
  </tr>
  <tr>
    <td style="text-align: center;"><i>text 중복 데이터</i></td>
  </tr>
</table>

   - 동일한 `text`가 서로 다른 `category`를 가짐 → 신뢰도 저하

<table style="margin: 0; margin-bottom: 20px;">
  <tr>
    <td><img src="https://john-yim.github.io/portfolio/images/category_dup.png" style="width: 700px; height: auto;"></td>
  </tr>
  <tr>
    <td style="text-align: center;"><i>text중복 데이터 중 category의 고유값이 2개 이상인 데이터</i></td>
  </tr>
</table>

   → `최빈 category` 기준으로 정제

3. **Validation 데이터셋 비율 불균형**
   - 전처리 과정에서 Validation 데이터의 비율이 지나치게 작아져, 모델 성능 평가의 신뢰도가 낮아지는 문제가 발생.
<table style="margin: 0; margin-bottom: 20px;">
  <tr>
    <td><img src="https://john-yim.github.io/portfolio/images/before_merge_ratio.png" style="width: 700px; height: auto;"></td>
  </tr>
  <tr>
    <td style="text-align: center;"><i>병합 전 Train/Validation 비율</i></td>
  </tr>
</table>
   → Train 데이터에 10%를 기존 Validation 데이터에 병합(stratify옵션을 사용하여 클래스 비율 유지한 채로 분할)
<table style="margin: 0; margin-bottom: 20px;">
  <tr>
    <td><img src="https://john-yim.github.io/portfolio/images/after_merge_ratio.png" style="width: 700px; height: auto;"></td>
  </tr>
  <tr>
    <td style="text-align: center;"><i>병합 후 Train/Validation 비율</i></td>
  </tr>
</table>

3. **Baseline 모델(LightGBM) 성능 저조**
   - 정확도 61%, 클래스 간 혼동 큼
   → 전이학습 기반 모델로 전환

4. **전이학습 모델 선정 과정**
   - KoBERT, KcBERT, KcELECTRA 비교
   - KcELECTRA가 가장 빠르고 성능 유사 → 최종 채택
   - Epoch 100, Patience 10 설정 → 최종 정확도 94% 달성

5. **2차 분류 시도 중 문제점 발견 (department 불균형)**
   - `val`에만 존재하는 부서가 다수
   - `text` 내 주소 정보가 `@#주소@#` 등으로 블러 처리되어 있어 부서 매핑 불가
   → department 분류 모델 제외

6. **MCP 알고리즘 개발**
   - 예측된 category를 기반으로 중앙정부 부처 정보(JSON)와 매핑
   - 홈페이지 / 전화번호 자동 제공
   - Flask 웹서버에 모델과 함께 통합 예정

---

## 🧠 성과 및 고찰

- 정확도 94% 이상의 실용 가능한 텍스트 분류 모델 구축
- 데이터 품질 개선이 모델 성능 향상에 핵심적 역할
- 단순 분류에서 나아가 실제 **정부 민원 안내 시스템에 활용 가능한 구조 설계**
- 현재 Flask 기반 웹 배포 및 부처 정보 연결 기능 통합 중
- 향후 지역 주소 기반 부서 분류 확장, LLM 기반 민원 응답 자동화 가능성 확인